# GWP Submission 3: Pricing American Options

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from scipy.stats import bernoulli
from scipy.optimize import fsolve
from scipy.stats import norm
import pandas as pd
from matplotlib import colors
import seaborn as sns
from pandas import DataFrame
import math
from scipy.stats import norm
import p4f as p4f
import sst
import scipy as sp
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

Write code to price an American Call option:

    Group Number = 25
    𝑢 = (1.10 + Group Number/100)

In [ ]:
# Our initial stock price
S0 = 95
# Time to expiration, the we rescale it to time per layer step
t = 1
# Strike price and risk-free rate
K = 90
r = 0
# Calculate the size of an up move, down move, and probability
Gn = 25 # Group Number
u = (1.10 + Gn/100)
d = 1/u
p = (np.exp(r * t) - d) / (u - d)
q = 1-p

In [ ]:
def binom_tree_call(N, T, S0, r, K, array_out=False):
    # Init
    dt = T/N
    u = (1.10 + 25/100)**(1/2)
    d = 1/u
    p = (np.exp(r*dt)-d)/(u-d)
    
    #Price Tree
    price_tree = np.zeros([N+1, N+1])
    
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j, i] = S0*(d**j)*(u**(i-j))
            
    # Option Value
    option = np.zeros([N+1, N+1])
    option[:, N] = np.maximum(np.zeros(N+1), price_tree[:, N]-K)
    
    # Calculate option price at t=0
    for i in np.arange(N-1, -1, -1):
        for j in np.arange(0, i+1):
            option[j, i] = np.exp(-r*dt)*(p*option[j, i+1]+(1-p)*option[j+1, i+1])
            
    #Return
    if array_out:
        return [option[0, 0], price_tree, option]
    else:
        return option[0, 0]

* The value of the derivative at each node

In [ ]:
binom_tree_call(2, 1, 95, 0, 90, True)

Pricing an American Put Option

In [ ]:
def binom_tree_call_put(N, T, SO, r, K, call=True, array_out=False):
    # Init
    dt = T/N
    u = (1.10 + 25/100)**(1/2)
    d = 1/u
    p = (np.exp(r*dt)-d)/(u-d)
    
    #Price Tree
    price_tree = np.zeros([N+1, N+1])
    
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j, i] = S0*(d**j)*(u**(i-j))
            
    # Option Value
    option = np.zeros([N+1, N+1])
    if call:
        option[:, N] = np.maximum(np.zeros(N+1), price_tree[:, N]-K)
    else:
        option[:, N] = np.maximum(np.zeros(N+1), K-price_tree[:, N])
    
    # Calculate option price at t=0
    for i in np.arange(N-1, -1, -1):
        for j in np.arange(0, i+1):
            option[j, i] = np.exp(-r*dt)*(p*option[j, i+1]+(1-p)*option[j+1, i+1])
            
    #Return
    if array_out:
        return [option[0, 0], price_tree, option]
    else:
        return option[0, 0]

* The value of the derivative at each node

In [ ]:
binom_tree_call_put(N=2, T=1, SO=95, r=0, K=90, call=False, array_out=True)

* Checking for early exercise

In [ ]:
print(p)
print(q)

In [ ]:
R = 1 + r
early_exercise = 1/R*(q*0+(1-q)*19.63)
early_exercise

Pricing an exotic option with European payoff, Up-and-Out (UAO) European Call Option

* ST - K if MAX St =< B,
* 0 if MAX St> B

Therefore if price is above 130 then we consider the option price is 0.


In [ ]:
binom_tree_call(2, 1, 100, 0, 90, True)